# Spark SQL Examples

Run the code cells below. This is the same code from the previous screencast.

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum

import datetime

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

In [0]:
path = "/FileStore/sparkify_log_small.json"
user_log = spark.read.json(path)

In [0]:
user_log.take(1)

In [0]:
display(user_log.printSchema())

# Create a View And Run Queries

The code below creates a temporary view against which you can run SQL queries.

Note the different ways you can run SQL queries in a notebook!

In [0]:
user_log.createOrReplaceTempView("user_log_table")

In [0]:
display(spark.sql("SELECT * FROM user_log_table LIMIT 2"))

In [0]:
%sql
--Note the line above changes this to a SQL cell so you can write SQL directly in this cell

SELECT * 
FROM user_log_table 
LIMIT 2
          

In [0]:
%sql 

SELECT COUNT(*) 
FROM user_log_table 


In [0]:
%sql
SELECT userID, firstname, page, song
FROM user_log_table 
WHERE userID == '1046'

In [0]:
%sql

SELECT DISTINCT page
FROM user_log_table 
ORDER BY page ASC
         

# User Defined Functions

In [0]:
spark.udf.register("get_hour", lambda x: int(datetime.datetime.fromtimestamp(x / 1000.0).hour) if not x is None else 0, IntegerType())

In [0]:
%sql 

SELECT *, get_hour(ts) AS hour
FROM user_log_table 
LIMIT 1


In [0]:
songs_in_hour = spark.sql('''
          SELECT get_hour(ts) AS hour, COUNT(*) as plays_per_hour
          FROM user_log_table
          WHERE page = "NextSong"
          GROUP BY hour
          ORDER BY cast(hour as int) ASC
          '''
          )

In [0]:
songs_in_hour.show()

# Converting Results to Pandas

In [0]:
songs_in_hour_pd = songs_in_hour.toPandas()

In [0]:
display(songs_in_hour_pd)